In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
df =pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

1= Fraud ,
0= normal transaction 

In [6]:
df["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

**preprocessing data**

In [7]:
#separate the data
nt=df[df["Class"]==0]
ft=df[df["Class"]==1]

In [8]:
nt.shape

(284315, 31)

In [9]:
ft.shape

(492, 31)

In [10]:
nt["Amount"].describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [11]:
ft["Amount"].describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [12]:
df_class=df.groupby("Class").mean()

In [13]:
df_class

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under sampling the dataset 

In [14]:
nt=nt.sample(n=492)

In [15]:
new_df = pd.concat([nt, ft], axis=0)

In [16]:
new_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
265421,161877.0,1.227499,-1.693221,-0.744231,0.902453,-1.286335,-0.631515,-0.118420,-0.125958,1.195359,...,0.414974,0.372978,-0.153797,0.065462,-0.481362,0.315983,-0.097225,0.022601,405.00,0
232972,147391.0,2.041198,-0.218276,-1.405547,0.187570,0.001916,-0.781668,-0.043787,-0.090115,0.677407,...,0.273900,0.844092,-0.023770,-0.407890,0.190140,-0.089576,-0.029237,-0.075626,1.00,0
110850,71999.0,1.272090,-0.207350,-0.226690,-0.478389,0.015834,-0.134279,-0.078940,-0.067544,0.290997,...,-0.016195,-0.052330,-0.159845,-0.856115,0.351511,1.541157,-0.105145,-0.012500,39.99,0
168812,119395.0,1.423686,-0.697396,-2.285343,1.279685,0.566797,-0.581887,1.051402,-0.338990,-0.428299,...,0.361313,0.367482,-0.252680,0.712799,0.321087,-0.567056,-0.098176,-0.012910,316.96,0
142547,84783.0,-0.961905,0.660210,2.020333,0.741632,0.790889,-0.159196,0.773506,-0.160110,-0.218417,...,-0.085387,0.044290,-0.390744,0.097777,0.540312,-0.319564,-0.215023,-0.130371,5.56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [17]:
new_df.Class.value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [18]:
new_df_gb=new_df.groupby("Class").mean()

In [19]:
new_df_gb

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,91652.554878,-0.031150,-0.013942,0.075011,0.023852,-0.049860,0.181447,0.048632,-0.075187,0.089065,...,0.002742,0.084557,0.040603,0.016955,-0.025729,-0.005035,-0.005258,0.014873,-0.016233,107.415366
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [20]:
new_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
265421,161877.0,1.227499,-1.693221,-0.744231,0.902453,-1.286335,-0.631515,-0.118420,-0.125958,1.195359,...,0.414974,0.372978,-0.153797,0.065462,-0.481362,0.315983,-0.097225,0.022601,405.00,0
232972,147391.0,2.041198,-0.218276,-1.405547,0.187570,0.001916,-0.781668,-0.043787,-0.090115,0.677407,...,0.273900,0.844092,-0.023770,-0.407890,0.190140,-0.089576,-0.029237,-0.075626,1.00,0
110850,71999.0,1.272090,-0.207350,-0.226690,-0.478389,0.015834,-0.134279,-0.078940,-0.067544,0.290997,...,-0.016195,-0.052330,-0.159845,-0.856115,0.351511,1.541157,-0.105145,-0.012500,39.99,0
168812,119395.0,1.423686,-0.697396,-2.285343,1.279685,0.566797,-0.581887,1.051402,-0.338990,-0.428299,...,0.361313,0.367482,-0.252680,0.712799,0.321087,-0.567056,-0.098176,-0.012910,316.96,0
142547,84783.0,-0.961905,0.660210,2.020333,0.741632,0.790889,-0.159196,0.773506,-0.160110,-0.218417,...,-0.085387,0.044290,-0.390744,0.097777,0.540312,-0.319564,-0.215023,-0.130371,5.56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


Spliting the data into features and target

In [21]:
x=new_df.drop(columns='Class',axis=1)
y=new_df["Class"]

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [23]:
print(x.shape, x_train.shape, x_test.shape)

(984, 30) (787, 30) (197, 30)


In [24]:
model = LogisticRegression()

In [25]:
model.fit(x_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
# accuracy on training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [27]:
print(f'Accuracy on Training data : , {training_data_accuracy}')


Accuracy on Training data : , 0.9415501905972046


In [28]:
# accuracy on test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [29]:
print(f'Accuracy score on Test Data : , {test_data_accuracy}')

Accuracy score on Test Data : , 0.9086294416243654


In [30]:
min_values = new_df.describe().loc['min']
max_values = new_df.describe().loc['max']

print("Minimum Values:\n", min_values)
print("\nMaximum Values:\n", max_values)

Minimum Values:
 Time      406.000000
V1        -30.552380
V2        -19.230653
V3        -31.103685
V4         -4.381252
V5        -22.105532
V6         -7.216809
V7        -43.557242
V8        -41.044261
V9        -13.434066
V10       -24.588262
V11        -2.439501
V12       -18.683715
V13        -3.127795
V14       -19.214325
V15        -4.498945
V16       -14.129855
V17       -25.162799
V18        -9.498746
V19        -3.681904
V20        -4.128186
V21       -22.797604
V22        -8.887017
V23       -19.254328
V24        -2.028024
V25        -4.781606
V26        -1.152671
V27        -7.263482
V28        -4.455921
Amount      0.000000
Class       0.000000
Name: min, dtype: float64

Maximum Values:
 Time      171858.000000
V1             2.256732
V2            22.057729
V3             3.511429
V4            12.114672
V5            11.095089
V6            12.128950
V7            26.237722
V8            20.007208
V9             6.956877
V10           10.908217
V11           12.018913


In [31]:
# Define the minimum and maximum values for each feature
min_values = {
    'Time': 266.000000, 'V1': -30.552380, 'V2': -14.505648, 'V3': -31.103685, 'V4': -4.399240, 
    'V5': -22.105532, 'V6': -10.136371, 'V7': -43.557242, 'V8': -41.044261, 'V9': -13.434066, 
    'V10': -24.588262, 'V11': -2.759528, 'V12': -18.683715, 'V13': -3.127795, 'V14': -19.214325, 
    'V15': -4.498945, 'V16': -14.129855, 'V17': -25.162799, 'V18': -9.498746, 'V19': -3.681904, 
    'V20': -7.642808, 'V21': -22.797604, 'V22': -8.887017, 'V23': -19.254328, 'V24': -2.100164, 
    'V25': -4.781606, 'V26': -1.443139, 'V27': -7.263482, 'V28': -8.307955, 'Amount': 0.000000
}

max_values = {
    'Time': 172575.000000, 'V1': 2.353031, 'V2': 22.057729, 'V3': 3.358655, 'V4': 12.114672, 
    'V5': 13.367253, 'V6': 6.474115, 'V7': 5.802537, 'V8': 20.007208, 'V9': 4.619855, 
    'V10': 6.982557, 'V11': 12.018913, 'V12': 2.660866, 'V13': 3.494972, 'V14': 4.047301, 
    'V15': 2.471358, 'V16': 3.139656, 'V17': 6.739384, 'V18': 3.790316, 'V19': 5.228342, 
    'V20': 11.059004, 'V21': 27.202839, 'V22': 8.361985, 'V23': 5.466230, 'V24': 1.237015, 
    'V25': 2.208209, 'V26': 2.745261, 'V27': 4.610936, 'V28': 4.106596, 'Amount': 2290.050000
}


In [32]:
# Function to generate a random number between min and max values
def generate_random_data(min_vals, max_vals):
    data = {feature: np.random.uniform(min_vals[feature], max_vals[feature]) for feature in min_vals}
    return pd.DataFrame([data])

In [33]:
new_data_df = generate_random_data(min_values, max_values)

In [34]:
prediction = model.predict(new_data_df)

# Print the prediction
if prediction[0] == 1:
    print("The transaction is fraudulent.")
else:
    print("The transaction is not fraudulent.")

The transaction is fraudulent.
